In [37]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
import numpy as np
import pandas as pd
import random, re

# Part 3: Clustering, Modeling, and Solo Generation
## Retrieving S-expressions
We read in the csv's generated by Part 2

In [38]:
labels_path = 'test_files/s_exp.csv'
features_path = 'test_files/s_exp_features.csv'

In [39]:
s_exp_labels = pd.read_csv(labels_path, index_col=0)
s_exp_labels.head()

,exp,song_id,song_index
0,0 4 X|0.125|0.155 X|0.290|0.725,0,0
1,0 0 L|0.678|0.340,0,1
2,-3 0 X|0.036|0.174 X|0.255|0.164 C|0.443|0.195...,0,2
3,0 0 C|0.004|0.243,0,3
4,-3 7 A|0.171|0.073 C|0.263|0.130 X|0.408|0.185...,0,4


In [40]:
s_exp_features = pd.read_csv(features_path, index_col = 0)
s_exp_features.head()

,0,1,2,3,4,5
0,2.0,0.125000,0.584635,4.0,0.0,0.0880
1,1.0,0.678385,-0.678385,0.0,0.0,0.1360
2,5.0,0.036458,-11.315104,1.5,1.0,0.3913
3,1.0,0.003906,-2.003906,0.0,0.0,0.1944
4,7.0,0.170573,-31.157552,4.0,2.0,0.5759


## Copied Note Functionality

In [41]:
notes = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
note_to_num = dict([[n, i] for i, n in enumerate(notes)])
num_to_note = dict([[v, k] for k, v in note_to_num.items()])
same_note = {'A#':'Bb', 'C#':'Db', 'D#':'Eb', 'F#': 'Gb', 'G#':'Ab'}

# checks if a note is formatted correctly and splits it into its component parts
def split_note(note):
    assert re.fullmatch('[A-G][#|b]?[0-7]', note) is not None, 'Note \'%s\' not formatted correctly.'%note
    note, octave = note[:-1], int(note[-1])
    if note in same_note:
        note = same_note[note]
    return note, octave

# shifts the note by amount half-steps (possibly negative)
def shift_note(note, amount):
    note, octave = split_note(note)
    new_num = note_to_num[note] + amount
    if new_num > 11:
        octave += 1
    elif new_num < 0:
        octave -= 1
    return num_to_note[(new_num) % 12] + str(octave)

def get_root(chord):
    r = re.findall('^[A-G][#|b]?', chord) 
    assert r is not None, 'Chord \'%s\'does not contain root note'%chord
    return r[0]

# output is positive if note2 is above noteorchord1, 0 if same
def find_note_dist(note_or_chord1, note2, chord=False):
    note1 = '%s0'%get_root(note_or_chord1) if chord else note_or_chord1
    note1, octave1 = split_note(note1)
    note2, octave2 = split_note(note2)
    dist = (octave2 - octave1) * 12 + note_to_num[note2] - note_to_num[note1]
    return dist % 12 if chord else dist

chord_dictionary = {
    "major": {"C": [0, 4, 7], "L": [2, 4, 6, 11]},
    "minor": {"C": [0, 4, 8], "L": [2, 3, 5, 7, 10]},
    "augmented": {"C": [0, 4, 8], "L": []},
    "diminished": {"C": [0, 3, 6] ,  "L": []},
    "half-diminished": {"C": [0, 3, 6, 10], "L": []},
    "dominant-seventh": {"C": [0, 4, 7, 10], "L": [] }
}

def find_chord_type(chord):
    if "m7b5" in chord:
        return "half-diminished"
    elif "j7" in chord:
        return "dominant-seventh"
    elif "o" in chord:
        return "diminished"
    elif "m" in chord: 
        return "minor"
    else:
        return "major"

## K-Means
The idea is to group up S-expressions by how similar they are, and put similar S-expressions into a single "node". See Part 2 for what features we use to determine similarity.
### Raw Clustering
We choose the number of clusters proportionally to how many S-expressions we want to cluster. We may experiment with this ratio to see how it affects the generated solo.
We use sklearn's K-Means function to do the clustering.
The features DataFrame may have any number of columns (features).

In [42]:
exp_to_cluster_ratio = 4
k_size = int(len(s_exp_features) / exp_to_cluster_ratio)
k_size

23

In [43]:
def get_kmeans_clusters(s_exp_features):
    features_scaled = preprocessing.scale(s_exp_features)
    kmeans = KMeans(n_clusters=k_size).fit(features_scaled)
    return kmeans.labels_

In [44]:
get_kmeans_clusters(s_exp_features)

array([14,  5,  9,  1, 19,  9,  2,  3, 19, 14,  9,  9,  1, 12,  9,  5, 13,
       18,  1, 12,  9, 19,  2,  3,  2,  2, 13,  3,  2, 15,  5, 12, 11,  1,
       10,  7,  2,  8, 11,  4,  5, 11, 20,  1,  2,  5, 17, 11,  8,  5, 10,
        4, 20, 18, 20,  7,  3, 11,  0,  8,  6,  6,  8,  7,  4,  1, 21, 11,
        5,  0, 20,  3, 16, 22, 10,  8,  5, 21,  8, 12, 20, 15,  8, 15, 21,
       21,  6,  8, 15, 22,  1, 21], dtype=int32)

### Creating Node Objects
Node objects include a unique label, a list of s-expressions, and a conditional probability table (generated next) describing which node is likely to follow.

In [45]:
class Node:
    def __init__(self, node_num):
        self.node_num = node_num
        self.s_exp = [] # list of s-exp-ids
        self.cpt = {} # maps from node_num to conditional probability
    
    def add_exp(self, s):
        self.s_exp.append(s)

In [46]:
def generate_nodes(labels):
    node_objects = [Node(i) for i in range(k_size)] #list of nodes for the Markov chain

    for i, label in enumerate(labels):
        cluster_num = label
        node_objects[label].add_exp(i)
    return node_objects

## Markov Chaining
### Generating CPTs

In [47]:
def generate_cpt(node_objects):
    for outer_node in node_objects:
        outer_node_count = 0
        for inner_node in node_objects:
            outer_node.cpt[inner_node.node_num] = 0.0
            for outer_id in outer_node.s_exp:
                for inner_id in inner_node.s_exp:
                    outer_exp, inner_exp = s_exp_labels.loc[outer_id], s_exp_labels.loc[inner_id]
                    if outer_exp['song_id'] == inner_exp['song_id'] and \
                       inner_exp['song_index'] - outer_exp['song_index'] == 1:
                            outer_node_count += 1
                            outer_node.cpt[inner_node.node_num] += 1
        if outer_node_count:
            outer_node.cpt = {k: (v / outer_node_count) for k, v in outer_node.cpt.items()}

### Generating a Probabilistic Sequence of S-expressions

In [48]:
def select_from_weighted_dct(dct):
    rand = random.random() #random value between 0 and 1
    total = 0
    for k, v in dct.items():
        total += v
        if rand <= total: #if running total exceeds probability, that's what you want
            return k

In [49]:
def sequence_s_expressions(n, node_objects):
    s_exp_ids = []
    next_node = node_objects[random.randint(0, len(node_objects))] # random start node
    for i in range(n):
        next_node_num = select_from_weighted_dct(next_node.cpt)
        next_node = node_objects[next_node_num]
        next_s_exp_id = random.choice(next_node.s_exp)
        s_exp_ids.append(next_s_exp_id) #store id in the list
    return s_exp_ids

## Producing Notes
### Selecting Notes from S-expressions

In [50]:
def get_notes_from_category(chord, category):
    if category == 'H' and category == 'A':
        category = random.choice(['C', 'L'])
    possible_notes = []
    root = get_root(chord) + '4'
    chord_type = find_chord_type(chord)
    if category == 'X':
        intervals = [i for i in range(12)]
    else:
        intervals = chord_dictionary[chord_type][category]
    for i in intervals: 
        possible_notes.append(shift_note(root, i)[:-1])
    return possible_notes

In [51]:
def select_note(lst, curr, min_s, max_s): # maybe split into two functions?
    '''
    lst: output of the possible notes (letter only, no octave)
    curr: current note ('C4')
    min_s: minimum slope
    max_s: maximum slope
    
    return: single full note from slope-filtered list; else note from unfiltered list
    '''
    master_lst = []
    octave = split_note(curr)[1]
    all_possible = [note + str(octave + offset) for offset in [-1,0,1] for note in lst]
    for value in all_possible:
        dist = find_note_dist(curr, value)
        if dist >= min_s and dist <= max_s:
            master_lst.append(value)
    if master_lst:
        return random.choice(master_lst)
    if lst:
        return random.choice(lst) + str(octave)
    return 'A0'

In [52]:
def produce_notes(num_measures, list_of_chords, node_objects): #length of chords list = num of measures
    notes_df = pd.DataFrame(columns=['note_name', 'start_time', 'duration'])
    s_exp_ids = sequence_s_expressions(num_measures, node_objects)
    curr_note = 'Bb3' # TODO: figure out how to determine the initial note
    for i in range(num_measures): #i refers to measure number
        s_exp = s_exp_labels.loc[s_exp_ids[i], 'exp'] #ith s-exp, a string
        split_list = s_exp.split(' ')
        min_slope, max_slope = int(split_list[0]), int(split_list[1])
        for term in split_list[2:]:
            elements = term.split("|")
            category, start, duration = elements[0], elements[1], elements[2]
            poss_notes_list = get_notes_from_category(list_of_chords[i], category)
            selected_note = select_note(poss_notes_list, curr_note, min_slope, max_slope)
            new_row = {'note_name': selected_note, 'start_time': float(start) + i, 'duration': duration}
            notes_df = notes_df.append(new_row, ignore_index=True)
            curr_note = selected_note
    return notes_df

### Generating a Solo

In [53]:
nodes = generate_nodes(get_kmeans_clusters(s_exp_features))
generate_cpt(nodes)
notes_df = produce_notes(2, ['Bb', 'D'], nodes)
notes_df

KeyError: 'A'